# 网络效应：当用户之间存在相互影响

**预计学习时间：60-90 分钟**

---

## 学习目标

完成本 Notebook 后，你将能够：

1. 理解 SUTVA 假设及其违背情况
2. 掌握处理网络效应的实验设计方法
3. 学会估计直接效应和溢出效应
4. 应用聚类随机化和 Ego-cluster 方法
5. 分析真实业务场景中的网络效应问题

---

## 引入场景：微信朋友圈的新功能实验

想象你是微信的产品经理，准备测试一个新功能：**「朋友圈视频点赞特效」**。

你随机选择 50% 的用户开启这个功能，另外 50% 作为对照组。实验运行一周后，你发现：

- ✅ **实验组用户**的点赞率提升了 15%
- 🤔 **对照组用户**的点赞率也提升了 8%

**这怎么可能？对照组用户根本没看到新功能啊！**

答案就在**网络效应**中：
- 实验组用户 A 看到新功能，点赞更频繁
- A 的好友 B（在对照组）看到 A 的点赞，也更愿意点赞
- 实验效应通过社交网络"传染"到了对照组

这违背了因果推断中的一个关键假设：**SUTVA**（Stable Unit Treatment Value Assumption）。

今天我们就来学习如何处理这类问题。

---

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import networkx as nx
from scipy import stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 默认配置
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✅ 环境准备完成")

---

## Part 1: SUTVA 假设 - 因果推断的"独立性假设"

### 1.1 什么是 SUTVA？

**SUTVA** (Stable Unit Treatment Value Assumption) 包含两部分：

1. **无干扰假设 (No Interference)**：
   - 用户 $i$ 的结果只取决于 $i$ 自己是否接受处理
   - 不受其他用户是否接受处理的影响

2. **一致性假设 (Consistency)**：
   - 处理的形式是唯一的
   - 不存在多个版本的"同一个处理"

用数学语言表达：

$$
Y_i(T_i, T_{-i}) = Y_i(T_i)
$$

其中：
- $Y_i$: 用户 $i$ 的结果
- $T_i$: 用户 $i$ 的处理分配
- $T_{-i}$: 其他所有用户的处理分配

**白话翻译**：你的结果只和你自己有关，与别人无关。

---

### 1.2 SUTVA 违背的场景

在互联网产品中，SUTVA 经常被违背：

| 场景类型 | 具体案例 | 网络效应机制 |
|---------|---------|-------------|
| **社交网络** | 微信朋友圈、Instagram | 好友的行为影响我的行为 |
| **双边市场** | Uber、外卖平台 | 司机/骑手是共享资源 |
| **共享资源** | 优惠券池、库存 | 实验组多用，对照组少得 |
| **市场竞争** | 广告竞价、搜索排名 | 实验组出价高，对照组排名降低 |
| **传染病** | 疫苗实验 | 免疫屏障保护未接种者 |

---

### 1.3 可视化：网络效应的传播

让我们模拟一个社交网络，看看处理效应如何"传染"。

In [ ]:
def create_social_network(n_users=50, avg_friends=5):
    """
    创建一个社交网络图
    
    Parameters:
    - n_users: 用户数量
    - avg_friends: 平均好友数
    
    Returns:
    - G: NetworkX 图对象
    """
    # 使用 Barabási-Albert 模型生成社交网络（符合"富者更富"特性）
    G = nx.barabasi_albert_graph(n_users, avg_friends // 2)
    return G

def simulate_network_experiment(G, treatment_prob=0.5, direct_effect=0.3, spillover_effect=0.15):
    """
    模拟有网络效应的实验
    
    Parameters:
    - G: 社交网络图
    - treatment_prob: 处理分配概率
    - direct_effect: 直接效应（实验组用户自己受到的影响）
    - spillover_effect: 溢出效应（每个实验组好友的影响）
    
    Returns:
    - df: 包含实验数据的 DataFrame
    """
    n_users = G.number_of_nodes()
    
    # 随机分配处理
    treatment = np.random.binomial(1, treatment_prob, n_users)
    
    # 基线结果（无处理情况下的结果）
    baseline = np.random.normal(0.5, 0.1, n_users)
    
    # 计算每个用户的实验组好友比例
    treated_friend_ratio = np.zeros(n_users)
    for i in range(n_users):
        neighbors = list(G.neighbors(i))
        if len(neighbors) > 0:
            treated_friend_ratio[i] = treatment[neighbors].mean()
    
    # 结果 = 基线 + 直接效应 + 溢出效应
    outcome = baseline + treatment * direct_effect + treated_friend_ratio * spillover_effect
    
    # 添加噪声
    outcome += np.random.normal(0, 0.05, n_users)
    
    return pd.DataFrame({
        'user_id': range(n_users),
        'treatment': treatment,
        'treated_friend_ratio': treated_friend_ratio,
        'outcome': outcome,
        'baseline': baseline
    })

# 创建网络并运行实验
G = create_social_network(n_users=100, avg_friends=8)
df_network = simulate_network_experiment(G, direct_effect=0.3, spillover_effect=0.15)

print(f"📊 网络统计：")
print(f"   - 用户数量: {G.number_of_nodes()}")
print(f"   - 好友关系数: {G.number_of_edges()}")
print(f"   - 平均好友数: {2 * G.number_of_edges() / G.number_of_nodes():.1f}")
print(f"\n📊 实验分组：")
print(f"   - 实验组: {df_network['treatment'].sum()} 人")
print(f"   - 对照组: {(1 - df_network['treatment']).sum()} 人")

In [ ]:
# 可视化网络效应
def visualize_network_effect(G, df):
    """
    可视化社交网络和实验分组
    """
    # 计算布局
    pos = nx.spring_layout(G, k=0.5, iterations=50)
    
    # 准备边的坐标
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    # 准备节点的坐标和颜色
    node_x = [pos[i][0] for i in range(len(df))]
    node_y = [pos[i][1] for i in range(len(df))]
    node_color = df['treatment'].map({1: '实验组', 0: '对照组'})
    node_size = df['treated_friend_ratio'] * 30 + 10  # 大小表示实验组好友比例
    
    # 绘图
    fig = go.Figure()
    
    # 添加边
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        mode='lines',
        line=dict(width=0.5, color='#d3d3d3'),
        hoverinfo='none',
        showlegend=False
    ))
    
    # 添加节点
    for treatment_group, color in [(1, '#2D9CDB'), (0, '#EB5757')]:
        mask = df['treatment'] == treatment_group
        group_name = '实验组' if treatment_group == 1 else '对照组'
        
        fig.add_trace(go.Scatter(
            x=[node_x[i] for i in range(len(node_x)) if mask.iloc[i]],
            y=[node_y[i] for i in range(len(node_y)) if mask.iloc[i]],
            mode='markers',
            name=group_name,
            marker=dict(
                size=[node_size.iloc[i] for i in range(len(node_size)) if mask.iloc[i]],
                color=color,
                line=dict(width=1, color='white')
            ),
            text=[f"用户 {i}<br>实验组好友: {df['treated_friend_ratio'].iloc[i]:.0%}" 
                  for i in range(len(df)) if mask.iloc[i]],
            hoverinfo='text'
        ))
    
    fig.update_layout(
        title=dict(
            text="社交网络可视化<br><sub>节点大小表示实验组好友的比例</sub>",
            x=0.5,
            xanchor='center'
        ),
        showlegend=True,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=600,
        plot_bgcolor='white'
    )
    
    return fig

fig = visualize_network_effect(G, df_network)
fig.show()

print("\n💡 观察要点：")
print("   1. 蓝色节点（实验组）和红色节点（对照组）随机分布")
print("   2. 节点越大，表示该用户的实验组好友越多")
print("   3. 红色节点周围如果有很多蓝色节点，会受到溢出效应影响")

### 1.4 网络效应的影响：天真估计 vs 真实效应

In [ ]:
# 计算天真估计（忽略网络效应）
naive_ate = df_network[df_network['treatment'] == 1]['outcome'].mean() - \
            df_network[df_network['treatment'] == 0]['outcome'].mean()

# 真实的直接效应（我们设定的参数）
true_direct_effect = 0.3
true_spillover_effect = 0.15

# 对照组受到的平均溢出效应
control_spillover = df_network[df_network['treatment'] == 0]['treated_friend_ratio'].mean() * true_spillover_effect

print("📊 效应估计对比：")
print(f"\n1️⃣ 天真估计 (Naive ATE):")
print(f"   ATE = E[Y|T=1] - E[Y|T=0] = {naive_ate:.3f}")
print(f"\n2️⃣ 真实直接效应:")
print(f"   Direct Effect = {true_direct_effect:.3f}")
print(f"\n3️⃣ 对照组受到的溢出效应:")
print(f"   Spillover on Control = {control_spillover:.3f}")
print(f"\n⚠️  偏差分析:")
print(f"   估计偏差 = {naive_ate - true_direct_effect:.3f}")
print(f"   原因：对照组已经受到溢出效应影响，基线被"抬高"了")

# 可视化
fig = go.Figure()

categories = ['天真估计', '真实直接效应', '对照组<br>受到的溢出效应']
values = [naive_ate, true_direct_effect, control_spillover]
colors = ['#95A5A6', '#2D9CDB', '#F39C12']

fig.add_trace(go.Bar(
    x=categories,
    y=values,
    marker_color=colors,
    text=[f'{v:.3f}' for v in values],
    textposition='outside'
))

fig.update_layout(
    title="网络效应导致的估计偏差",
    yaxis_title="效应大小",
    height=400,
    showlegend=False
)

fig.show()

### 🤔 思考题 1

**问题**：在上面的例子中，天真估计为什么会低于真实直接效应？

**提示**：想想对照组的基线发生了什么变化。

<details>
<summary>点击查看答案</summary>

因为对照组用户也有实验组好友，受到了溢出效应的影响，基线被"抬高"了。

- 实验组：基线 + 直接效应 + 溢出效应
- 对照组：基线 + 溢出效应

天真估计 = (基线 + 直接效应 + 溢出效应) - (基线 + 溢出效应) < 直接效应

这就像在测试减肥药效果时，对照组的人也跟着实验组的人一起去健身房了。
</details>

---

## Part 2: 聚类随机化 (Cluster Randomization)

### 2.1 核心思想

**问题**：个体随机化会让实验组和对照组"混在一起"，导致溢出效应。

**解决方案**：以**群组**为单位进行随机化，让实验组和对照组"物理隔离"。

**生活类比**：
- ❌ **个体随机化**：在一个教室里，随机选一半学生发新教材，另一半用旧教材
  - 问题：用新教材的学生会和用旧教材的学生讨论，溢出效应严重
  
- ✅ **聚类随机化**：随机选一半班级发新教材，另一半班级用旧教材
  - 优势：不同班级之间互动少，溢出效应小

---

### 2.2 聚类的选择

**关键问题**：如何定义"群组"？

| 场景 | 聚类单位 | 理由 |
|-----|---------|-----|
| **社交网络** | 社区、学校、城市 | 地理隔离减少互动 |
| **双边市场 (Uber)** | 城市、区域 | 司机和乘客在同一区域内匹配 |
| **外卖平台** | 商圈 | 骑手和商家在同一商圈内服务 |
| **B2B SaaS** | 公司 | 同一公司的员工会相互影响 |

**选择原则**：
1. **Cluster 内部相互作用强**（同一个群组的用户互动多）
2. **Cluster 之间相互作用弱**（不同群组的用户互动少）
3. **Cluster 数量足够多**（至少 20-30 个，用于统计推断）

---

### 2.3 实现：地理聚类实验

In [ ]:
def simulate_cluster_experiment(n_clusters=30, users_per_cluster=50, 
                                direct_effect=0.3, within_spillover=0.2, between_spillover=0.02):
    """
    模拟聚类随机化实验
    
    Parameters:
    - n_clusters: 聚类数量（如城市数量）
    - users_per_cluster: 每个聚类的用户数
    - direct_effect: 直接效应
    - within_spillover: 聚类内溢出效应（同城用户之间）
    - between_spillover: 聚类间溢出效应（跨城用户之间）
    
    Returns:
    - df: 实验数据
    """
    # 聚类级别的随机化
    cluster_treatment = np.random.binomial(1, 0.5, n_clusters)
    
    data = []
    for cluster_id in range(n_clusters):
        # 当前聚类的处理状态
        is_treated_cluster = cluster_treatment[cluster_id]
        
        for user_id in range(users_per_cluster):
            # 基线结果
            baseline = np.random.normal(0.5, 0.1)
            
            # 直接效应
            direct = is_treated_cluster * direct_effect
            
            # 聚类内溢出效应（同城其他用户的影响）
            within_spill = is_treated_cluster * within_spillover * np.random.uniform(0.5, 1.0)
            
            # 聚类间溢出效应（跨城用户的影响，非常小）
            treated_clusters_ratio = cluster_treatment.mean()
            between_spill = treated_clusters_ratio * between_spillover
            
            # 最终结果
            outcome = baseline + direct + within_spill + between_spill + np.random.normal(0, 0.05)
            
            data.append({
                'cluster_id': cluster_id,
                'user_id': f'{cluster_id}_{user_id}',
                'cluster_treatment': is_treated_cluster,
                'outcome': outcome,
                'baseline': baseline
            })
    
    return pd.DataFrame(data)

# 运行聚类实验
df_cluster = simulate_cluster_experiment(
    n_clusters=30, 
    users_per_cluster=100,
    direct_effect=0.3,
    within_spillover=0.2,
    between_spillover=0.02
)

print("📊 聚类实验统计：")
print(f"   - 聚类数量: {df_cluster['cluster_id'].nunique()}")
print(f"   - 实验组聚类: {df_cluster.groupby('cluster_id')['cluster_treatment'].first().sum():.0f}")
print(f"   - 对照组聚类: {(1 - df_cluster.groupby('cluster_id')['cluster_treatment'].first()).sum():.0f}")
print(f"   - 总用户数: {len(df_cluster)}")

### 2.4 聚类分析：个体级 vs 聚类级

In [ ]:
# 方法 1: 天真的个体级分析（错误！）
naive_individual = df_cluster[df_cluster['cluster_treatment'] == 1]['outcome'].mean() - \
                   df_cluster[df_cluster['cluster_treatment'] == 0]['outcome'].mean()

# 标准误（错误地假设观测独立）
treated_std = df_cluster[df_cluster['cluster_treatment'] == 1]['outcome'].std()
control_std = df_cluster[df_cluster['cluster_treatment'] == 0]['outcome'].std()
n_treated = (df_cluster['cluster_treatment'] == 1).sum()
n_control = (df_cluster['cluster_treatment'] == 0).sum()
naive_se = np.sqrt(treated_std**2 / n_treated + control_std**2 / n_control)

# 方法 2: 正确的聚类级分析
cluster_means = df_cluster.groupby('cluster_id').agg({
    'cluster_treatment': 'first',
    'outcome': 'mean'
}).reset_index()

correct_cluster = cluster_means[cluster_means['cluster_treatment'] == 1]['outcome'].mean() - \
                  cluster_means[cluster_means['cluster_treatment'] == 0]['outcome'].mean()

# 聚类稳健标准误
cluster_treated_std = cluster_means[cluster_means['cluster_treatment'] == 1]['outcome'].std()
cluster_control_std = cluster_means[cluster_means['cluster_treatment'] == 0]['outcome'].std()
n_cluster_treated = (cluster_means['cluster_treatment'] == 1).sum()
n_cluster_control = (cluster_means['cluster_treatment'] == 0).sum()
cluster_se = np.sqrt(cluster_treated_std**2 / n_cluster_treated + cluster_control_std**2 / n_cluster_control)

print("📊 分析方法对比：")
print(f"\n1️⃣ 天真的个体级分析（错误）:")
print(f"   ATE = {naive_individual:.3f}")
print(f"   SE = {naive_se:.4f}")
print(f"   95% CI = [{naive_individual - 1.96*naive_se:.3f}, {naive_individual + 1.96*naive_se:.3f}]")
print(f"\n2️⃣ 正确的聚类级分析:")
print(f"   ATE = {correct_cluster:.3f}")
print(f"   SE = {cluster_se:.4f} (约为个体级的 {cluster_se/naive_se:.1f} 倍)")
print(f"   95% CI = [{correct_cluster - 1.96*cluster_se:.3f}, {correct_cluster + 1.96*cluster_se:.3f}]")
print(f"\n⚠️  关键差异:")
print(f"   聚类内的观测是相关的，不能当作独立样本！")
print(f"   正确的标准误要大得多，置信区间更宽。")

### 2.5 可视化：聚类内相关性 (Intra-Cluster Correlation)

In [ ]:
# 计算聚类内相关系数 (ICC)
def calculate_icc(df, cluster_col, outcome_col):
    """
    计算 Intra-Class Correlation Coefficient
    
    ICC = Var(cluster_mean) / [Var(cluster_mean) + Var(within_cluster)]
    """
    grand_mean = df[outcome_col].mean()
    cluster_means = df.groupby(cluster_col)[outcome_col].mean()
    
    # Between-cluster variance
    var_between = ((cluster_means - grand_mean) ** 2).mean()
    
    # Within-cluster variance
    var_within = df.groupby(cluster_col)[outcome_col].apply(lambda x: ((x - x.mean()) ** 2).mean()).mean()
    
    icc = var_between / (var_between + var_within)
    return icc

icc = calculate_icc(df_cluster, 'cluster_id', 'outcome')

print(f"📊 聚类内相关系数 (ICC) = {icc:.3f}")
print(f"\n💡 解读:")
print(f"   - ICC = 0: 聚类内用户完全独立")
print(f"   - ICC = 1: 聚类内用户完全相同")
print(f"   - 当前 ICC = {icc:.3f}: 聚类内用户有 {icc:.1%} 的相似性")

# 可视化不同聚类的结果分布
fig = go.Figure()

for cluster_id in sorted(df_cluster['cluster_id'].unique())[:10]:  # 只显示前10个聚类
    cluster_data = df_cluster[df_cluster['cluster_id'] == cluster_id]
    treatment_status = '实验组' if cluster_data['cluster_treatment'].iloc[0] == 1 else '对照组'
    color = '#2D9CDB' if cluster_data['cluster_treatment'].iloc[0] == 1 else '#EB5757'
    
    fig.add_trace(go.Box(
        y=cluster_data['outcome'],
        name=f'聚类 {cluster_id}<br>({treatment_status})',
        marker_color=color,
        boxmean='sd'
    ))

fig.update_layout(
    title=f"不同聚类的结果分布<br><sub>ICC = {icc:.3f}，聚类内用户结果相似</sub>",
    yaxis_title="结果指标",
    xaxis_title="聚类 ID",
    height=500,
    showlegend=False
)

fig.show()

### 2.6 示范：计算设计效应 (Design Effect)

**概念**：聚类随机化会降低统计效率，我们需要更大的样本量。设计效应量化了这种损失。

**公式**：
$$
\text{Design Effect} = 1 + (m - 1) \times \text{ICC}
$$

其中：
- $m$: 每个聚类的平均用户数
- ICC: 聚类内相关系数

**解读**：
- Design Effect = 2 意味着需要 2 倍的样本量才能达到同样的统计功效

下面我们实现设计效应的计算：

In [ ]:
# 📖 示范：计算设计效应
def calculate_design_effect(df, cluster_col, outcome_col):
    """
    计算设计效应
    
    Returns:
    - design_effect: 设计效应
    """
    # 1. 计算 ICC
    icc = calculate_icc(df, cluster_col, outcome_col)
    
    # 2. 计算每个聚类的平均用户数
    m = df.groupby(cluster_col).size().mean()
    
    # 3. 计算设计效应
    # Design Effect = 1 + (m - 1) × ICC
    design_effect = 1 + (m - 1) * icc
    
    return design_effect, icc, m

# 测试函数
de, icc, m = calculate_design_effect(df_cluster, 'cluster_id', 'outcome')

print(f"📊 设计效应计算：")
print(f"   - 聚类内相关系数 (ICC) = {icc:.3f}")
print(f"   - 每聚类平均用户数 (m) = {m:.0f}")
print(f"   - 设计效应 (DE) = {de:.2f}")
print(f"\n💡 这意味着：")
print(f"   聚类随机化需要 {de:.2f} 倍的样本量才能达到同样的统计功效")

---

## Part 3: Ego-Cluster 方法

### 3.1 动机：当无法做聚类随机化时

**问题**：有时我们无法进行聚类随机化：
- 聚类数量太少（如只有 5 个城市）
- 聚类之间差异太大（纽约 vs 小镇）
- 无法定义清晰的聚类边界（社交网络错综复杂）

**解决方案**：Ego-Cluster 方法
- 仍然进行个体随机化
- 但在分析时考虑每个用户的"自我网络" (Ego Network)

---

### 3.2 什么是 Ego Network？

**定义**：以某个用户（Ego）为中心的局部社交网络，包括：
- Ego：中心用户自己
- Alters：Ego 的直接好友
- Ties：Alters 之间的关系

**例子**：微信朋友圈
- Ego：你自己
- Alters：你的好友列表
- Ties：你的好友之间是否也是好友

---

### 3.3 Ego-Cluster 实验设计

**步骤**：
1. **个体随机化**：每个用户独立随机分配到实验组或对照组
2. **定义 Ego Network**：对每个用户，识别其直接好友
3. **计算暴露度**：每个用户的实验组好友比例
4. **估计效应**：
   - **直接效应**：用户自己接受处理的影响
   - **溢出效应**：用户好友接受处理的影响

In [ ]:
# 重新使用之前的社交网络数据
print("📊 Ego Network 分析：")
print(f"\n样本用户的 Ego Network 信息：")

# 选择几个样本用户分析其 Ego Network
sample_users = [0, 10, 20, 30]

for user_id in sample_users:
    neighbors = list(G.neighbors(user_id))
    n_friends = len(neighbors)
    n_treated_friends = df_network.loc[neighbors, 'treatment'].sum() if n_friends > 0 else 0
    
    print(f"\n用户 {user_id}:")
    print(f"  - 自己是: {'实验组' if df_network.loc[user_id, 'treatment'] == 1 else '对照组'}")
    print(f"  - 好友数量: {n_friends}")
    print(f"  - 实验组好友: {n_treated_friends} ({n_treated_friends/n_friends*100 if n_friends > 0 else 0:.0f}%)")
    print(f"  - 结果指标: {df_network.loc[user_id, 'outcome']:.3f}")

### 3.4 估计直接效应和溢出效应

**回归模型**：

$$
Y_i = \beta_0 + \beta_1 T_i + \beta_2 \bar{T}_{-i} + \epsilon_i
$$

其中：
- $Y_i$: 用户 $i$ 的结果
- $T_i$: 用户 $i$ 的处理状态（1 = 实验组，0 = 对照组）
- $\bar{T}_{-i}$: 用户 $i$ 的好友中实验组的比例（暴露度）
- $\beta_1$: **直接效应** (Direct Effect)
- $\beta_2$: **溢出效应** (Spillover Effect)

**解读**：
- $\beta_1$：我自己在实验组 vs 对照组的差异
- $\beta_2$：我的好友全在实验组 vs 全在对照组时，对我的影响

In [ ]:
from sklearn.linear_model import LinearRegression
from scipy import stats as sp_stats

# 准备回归数据
X = df_network[['treatment', 'treated_friend_ratio']].values
y = df_network['outcome'].values

# 拟合回归模型
model = LinearRegression()
model.fit(X, y)

# 获取系数
intercept = model.intercept_
beta_direct = model.coef_[0]  # 直接效应
beta_spillover = model.coef_[1]  # 溢出效应

# 计算标准误（简化版本）
y_pred = model.predict(X)
residuals = y - y_pred
mse = (residuals ** 2).mean()
n = len(y)
X_with_intercept = np.column_stack([np.ones(n), X])
var_coef = mse * np.linalg.inv(X_with_intercept.T @ X_with_intercept).diagonal()
se_direct = np.sqrt(var_coef[1])
se_spillover = np.sqrt(var_coef[2])

print("📊 Ego-Cluster 回归分析结果：")
print(f"\n回归方程: Y = {intercept:.3f} + {beta_direct:.3f}*T + {beta_spillover:.3f}*T_friends")
print(f"\n1️⃣ 直接效应 (β₁):")
print(f"   估计值 = {beta_direct:.3f}")
print(f"   标准误 = {se_direct:.4f}")
print(f"   95% CI = [{beta_direct - 1.96*se_direct:.3f}, {beta_direct + 1.96*se_direct:.3f}]")
print(f"   真实值 = 0.300 (我们设定的参数)")
print(f"\n2️⃣ 溢出效应 (β₂):")
print(f"   估计值 = {beta_spillover:.3f}")
print(f"   标准误 = {se_spillover:.4f}")
print(f"   95% CI = [{beta_spillover - 1.96*se_spillover:.3f}, {beta_spillover + 1.96*se_spillover:.3f}]")
print(f"   真实值 = 0.150 (我们设定的参数)")
print(f"\n💡 解读:")
print(f"   - 用户自己在实验组，结果平均提升 {beta_direct:.3f}")
print(f"   - 用户的好友全在实验组（vs 全在对照组），结果平均提升 {beta_spillover:.3f}")

### 3.5 可视化：直接效应 vs 溢出效应

In [ ]:
# 创建 2x2 对比图
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('直接效应', '溢出效应')
)

# 左图：直接效应（控制好友暴露度）
for treatment_val in [0, 1]:
    mask = df_network['treatment'] == treatment_val
    group_name = '实验组' if treatment_val == 1 else '对照组'
    color = '#2D9CDB' if treatment_val == 1 else '#EB5757'
    
    fig.add_trace(
        go.Box(
            y=df_network[mask]['outcome'],
            name=group_name,
            marker_color=color,
            boxmean='sd',
            showlegend=True
        ),
        row=1, col=1
    )

# 右图：溢出效应（分别看实验组和对照组）
for treatment_val, color, name in [(1, '#2D9CDB', '实验组用户'), (0, '#EB5757', '对照组用户')]:
    mask = df_network['treatment'] == treatment_val
    data = df_network[mask]
    
    fig.add_trace(
        go.Scatter(
            x=data['treated_friend_ratio'],
            y=data['outcome'],
            mode='markers',
            name=name,
            marker=dict(color=color, size=5, opacity=0.6),
            showlegend=True
        ),
        row=1, col=2
    )

# 添加回归线（溢出效应）
friend_ratio_range = np.linspace(0, 1, 100)
for treatment_val, color in [(1, '#2D9CDB'), (0, '#EB5757')]:
    y_pred = intercept + beta_direct * treatment_val + beta_spillover * friend_ratio_range
    
    fig.add_trace(
        go.Scatter(
            x=friend_ratio_range,
            y=y_pred,
            mode='lines',
            line=dict(color=color, width=2),
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="用户分组", row=1, col=1)
fig.update_xaxes(title_text="实验组好友比例", row=1, col=2)
fig.update_yaxes(title_text="结果指标", row=1, col=1)
fig.update_yaxes(title_text="结果指标", row=1, col=2)

fig.update_layout(
    title_text="直接效应 vs 溢出效应",
    height=500,
    showlegend=True
)

fig.show()

print("\n💡 图表解读：")
print("   左图：直接效应 = 实验组 vs 对照组的差异")
print("   右图：溢出效应 = 随着实验组好友比例增加，结果的变化趋势")
print("         注意两条线平行，斜率都是溢出效应，截距差是直接效应")

### 3.6 示范：计算总效应 (Total Effect)

**问题**：如果所有用户都接受处理（vs 都不接受），平均结果会提升多少？

**推导**：
- 当所有人都在实验组时：$T_i = 1$，$\bar{T}_{-i} = 1$
- 当所有人都在对照组时：$T_i = 0$，$\bar{T}_{-i} = 0$

$$
\text{Total Effect} = E[Y | T_i=1, \bar{T}_{-i}=1] - E[Y | T_i=0, \bar{T}_{-i}=0]
$$

代入回归方程 $Y = \beta_0 + \beta_1 T_i + \beta_2 \bar{T}_{-i}$：

$$
\text{Total Effect} = (\beta_0 + \beta_1 + \beta_2) - \beta_0 = \beta_1 + \beta_2
$$

下面我们实现总效应的计算：

In [ ]:
# 📖 示范：计算总效应
def calculate_total_effect(beta_direct, beta_spillover):
    """
    计算总效应
    
    Parameters:
    - beta_direct: 直接效应
    - beta_spillover: 溢出效应
    
    Returns:
    - total_effect: 总效应
    """
    # 当所有人都在实验组时: T_i = 1, T_friends = 1
    # 当所有人都在对照组时: T_i = 0, T_friends = 0
    # Total Effect = (β0 + β1*1 + β2*1) - (β0 + β1*0 + β2*0) = β1 + β2
    total_effect = beta_direct + beta_spillover
    
    return total_effect

# 测试函数
total_effect = calculate_total_effect(beta_direct, beta_spillover)

print("📊 效应分解：")
print(f"   - 直接效应: {beta_direct:.3f}")
print(f"   - 溢出效应: {beta_spillover:.3f}")
print(f"   - 总效应: {total_effect:.3f}")
print(f"\n💡 业务含义：")
print(f"   如果把新功能推广到所有用户，平均效果会提升 {total_effect:.3f}")
print(f"   其中 {beta_direct/total_effect*100:.0f}% 来自直接效应，{beta_spillover/total_effect*100:.0f}% 来自网络效应")

---

## Part 4: 溢出效应的深入分析

### 4.1 溢出效应的类型

| 溢出效应类型 | 机制 | 例子 |
|------------|------|------|
| **模仿效应** | 观察他人行为并模仿 | 看到朋友用新功能，自己也想试试 |
| **互补效应** | 他人使用提升我的收益 | 我的朋友都在用语音聊天，我也更愿意用 |
| **竞争效应** | 共享资源导致的负面影响 | 实验组用户多叫车，对照组用户等待时间变长 |
| **信息传播** | 通过社交网络传播信息 | 实验组用户分享内容，对照组用户看到 |

---

### 4.2 正向溢出 vs 负向溢出

**正向溢出**（正的 $\beta_2$）：
- 网络效应增强处理效果
- 例子：社交产品、通信工具

**负向溢出**（负的 $\beta_2$）：
- 共享资源被实验组抢占
- 例子：外卖平台（骑手是有限资源）、优惠券池

In [ ]:
# 模拟负向溢出：外卖平台实验
def simulate_negative_spillover(n_users=1000, treatment_prob=0.5, 
                                direct_effect=0.1, spillover_effect=-0.15):
    """
    模拟负向溢出效应：实验组用户获得优惠券，更频繁下单，
    导致骑手资源紧张，对照组用户配送时间变长
    """
    treatment = np.random.binomial(1, treatment_prob, n_users)
    
    # 整体实验组用户比例（影响资源紧张程度）
    overall_treatment_ratio = treatment.mean()
    
    # 基线：配送满意度
    baseline = np.random.normal(0.7, 0.1, n_users)
    
    # 结果
    outcome = baseline + \
              treatment * direct_effect + \
              overall_treatment_ratio * spillover_effect + \
              np.random.normal(0, 0.05, n_users)
    
    return pd.DataFrame({
        'treatment': treatment,
        'outcome': outcome
    })

df_negative = simulate_negative_spillover()

# 分析
treated_mean = df_negative[df_negative['treatment'] == 1]['outcome'].mean()
control_mean = df_negative[df_negative['treatment'] == 0]['outcome'].mean()
naive_effect = treated_mean - control_mean

print("📊 负向溢出案例：外卖优惠券实验")
print(f"\n设定参数：")
print(f"   - 直接效应（优惠券让实验组用户更满意）: +0.10")
print(f"   - 溢出效应（资源紧张降低所有人满意度）: -0.15")
print(f"\n观测结果：")
print(f"   - 实验组平均满意度: {treated_mean:.3f}")
print(f"   - 对照组平均满意度: {control_mean:.3f}")
print(f"   - 天真估计 ATE: {naive_effect:.3f}")
print(f"\n⚠️  陷阱:")
print(f"   天真估计 = 直接效应 - 溢出效应")
print(f"   实际上直接效应是 +0.10，但因为负向溢出，看起来更大！")
print(f"   如果全量推广，所有人都受到负向溢出，总效应可能是负的！")

### 4.3 识别溢出效应的方法

**方法 1: 对照组分层分析**
- 将对照组用户按"实验组好友比例"分层
- 看不同层的结果是否有差异
- 如果有差异，说明存在溢出效应

In [ ]:
# 对照组分层分析
control_group = df_network[df_network['treatment'] == 0].copy()

# 按实验组好友比例分层
control_group['exposure_bin'] = pd.cut(
    control_group['treated_friend_ratio'], 
    bins=[0, 0.3, 0.7, 1.0],
    labels=['低暴露 (<30%)', '中暴露 (30-70%)', '高暴露 (>70%)']
)

# 计算每层的平均结果
exposure_analysis = control_group.groupby('exposure_bin')['outcome'].agg(['mean', 'std', 'count']).reset_index()

print("📊 对照组分层分析（按实验组好友比例）：")
print(exposure_analysis.to_string(index=False))

# 可视化
fig = go.Figure()

fig.add_trace(go.Bar(
    x=exposure_analysis['exposure_bin'],
    y=exposure_analysis['mean'],
    error_y=dict(type='data', array=exposure_analysis['std'] / np.sqrt(exposure_analysis['count'])),
    marker_color='#EB5757',
    text=[f"{m:.3f}" for m in exposure_analysis['mean']],
    textposition='outside'
))

fig.update_layout(
    title="对照组用户的结果 vs 实验组好友比例<br><sub>如果存在溢出效应，应该看到递增趋势</sub>",
    xaxis_title="实验组好友比例",
    yaxis_title="平均结果",
    height=400
)

fig.show()

print("\n💡 解读：")
print("   对照组用户的结果随着实验组好友比例增加而提升")
print("   这是溢出效应的直接证据！")

### 4.4 示范：溢出效应的显著性检验

**问题**：如何检验溢出效应是否显著不为零？

**方法**：使用之前的回归模型，检验 $\beta_2$ 的 t 统计量。

**公式**：
$$
t = \frac{\hat{\beta}_2}{\text{SE}(\hat{\beta}_2)}
$$

**检验**：
- 原假设 $H_0: \beta_2 = 0$ (无溢出效应)
- 备择假设 $H_1: \beta_2 \neq 0$ (存在溢出效应)
- 使用双侧 t 检验

下面我们实现显著性检验：

In [ ]:
# 📖 示范：溢出效应的显著性检验
def test_spillover_significance(beta_spillover, se_spillover, alpha=0.05):
    """
    检验溢出效应是否显著
    
    Parameters:
    - beta_spillover: 溢出效应估计值
    - se_spillover: 标准误
    - alpha: 显著性水平
    
    Returns:
    - t_stat: t 统计量
    - p_value: p 值
    - is_significant: 是否显著
    """
    # 计算 t 统计量
    t_stat = beta_spillover / se_spillover
    
    # 计算 p 值（双侧检验）
    # 使用标准正态分布近似（大样本）
    p_value = 2 * (1 - sp_stats.norm.cdf(abs(t_stat)))
    
    # 判断是否显著
    is_significant = p_value < alpha
    
    return t_stat, p_value, is_significant

# 测试函数
t_stat, p_value, is_sig = test_spillover_significance(beta_spillover, se_spillover)

print("📊 溢出效应显著性检验：")
print(f"   - 估计值: {beta_spillover:.3f}")
print(f"   - 标准误: {se_spillover:.4f}")
print(f"   - t 统计量: {t_stat:.2f}")
print(f"   - p 值: {p_value:.4f}")
print(f"   - 结论: {'显著' if is_sig else '不显著'} (α=0.05)")

---

## Part 5: 业务案例分析

### 案例 1: 微信朋友圈视频点赞特效

**背景**：
- 产品：给视频点赞添加酷炫特效（类似抖音）
- 目标：提升用户互动率
- 挑战：社交网络产品，网络效应强

**实验设计**：
- ❌ **错误做法**：50% 用户随机开启新功能
  - 问题：实验组和对照组相互影响，效应被稀释
  
- ✅ **正确做法**：地理聚类随机化
  - 以城市为单位随机化
  - 实验组城市：所有用户都看到新特效
  - 对照组城市：所有用户都看不到新特效

**分析框架**：
1. 聚类级别的 ATE 估计
2. 聚类稳健标准误
3. 检查是否存在跨城溢出效应（通过地理位置检验）

In [ ]:
# 模拟案例 1
def simulate_wechat_experiment(n_cities=40, users_per_city=1000):
    """
    模拟微信朋友圈实验
    """
    city_treatment = np.random.binomial(1, 0.5, n_cities)
    
    data = []
    for city_id in range(n_cities):
        is_treated = city_treatment[city_id]
        
        # 城市级别的随机效应（有些城市用户更活跃）
        city_baseline = np.random.normal(0.15, 0.02)  # 点赞率基线
        
        for _ in range(users_per_city):
            # 用户级别的基线
            user_baseline = city_baseline + np.random.normal(0, 0.05)
            
            # 直接效应 + 网络效应
            direct_effect = is_treated * 0.03  # 新特效提升 3% 点赞率
            network_effect = is_treated * 0.02  # 网络效应额外提升 2%
            
            outcome = user_baseline + direct_effect + network_effect + np.random.normal(0, 0.01)
            
            data.append({
                'city_id': city_id,
                'city_treatment': is_treated,
                'like_rate': max(0, min(1, outcome))  # 限制在 [0, 1]
            })
    
    return pd.DataFrame(data)

df_wechat = simulate_wechat_experiment()

# 城市级别分析
city_stats = df_wechat.groupby('city_id').agg({
    'city_treatment': 'first',
    'like_rate': 'mean'
}).reset_index()

ate = city_stats[city_stats['city_treatment'] == 1]['like_rate'].mean() - \
      city_stats[city_stats['city_treatment'] == 0]['like_rate'].mean()

print("📊 微信朋友圈实验结果：")
print(f"\n实验设计：")
print(f"   - 城市数量: {df_wechat['city_id'].nunique()}")
print(f"   - 实验组城市: {(city_stats['city_treatment'] == 1).sum()}")
print(f"   - 对照组城市: {(city_stats['city_treatment'] == 0).sum()}")
print(f"\n结果：")
print(f"   - 实验组平均点赞率: {city_stats[city_stats['city_treatment'] == 1]['like_rate'].mean():.1%}")
print(f"   - 对照组平均点赞率: {city_stats[city_stats['city_treatment'] == 0]['like_rate'].mean():.1%}")
print(f"   - ATE: +{ate:.1%}")
print(f"\n💼 业务决策：")
print(f"   视频点赞特效使点赞率提升 {ate:.1%}，建议全量推广")

### 案例 2: 共享单车动态定价实验

**背景**：
- 产品：在高峰时段提高价格，引导用户避峰出行
- 目标：提升单车利用率和收入
- 挑战：单车是共享资源，存在负向溢出

**网络效应机制**：
- 实验组用户看到涨价，减少高峰时段用车
- 对照组用户发现单车更容易找到了（因为实验组用得少）
- 负向溢出：实验效应被低估

**实验设计**：
- 以商圈为单位随机化
- 确保不同商圈的单车供给相对独立

In [ ]:
# 模拟案例 2
def simulate_bike_pricing_experiment(n_areas=30, users_per_area=200):
    """
    模拟共享单车动态定价实验
    """
    area_treatment = np.random.binomial(1, 0.5, n_areas)
    
    data = []
    for area_id in range(n_areas):
        is_treated = area_treatment[area_id]
        
        # 基线骑行次数
        area_baseline = np.random.normal(10, 2)  # 每周平均骑行次数
        
        for _ in range(users_per_area):
            user_baseline = area_baseline + np.random.normal(0, 3)
            
            # 直接效应（涨价减少骑行）
            direct_effect = is_treated * (-2.0)  # 高峰涨价减少 2 次/周
            
            # 负向溢出（实验组用户少了，对照组用户发现车更好找）
            # 注意：这里的溢出是区域内的，不同区域独立
            spillover = is_treated * 0.5  # 车更好找，增加 0.5 次/周
            
            outcome = user_baseline + direct_effect + spillover + np.random.normal(0, 1)
            outcome = max(0, outcome)
            
            data.append({
                'area_id': area_id,
                'area_treatment': is_treated,
                'weekly_rides': outcome
            })
    
    return pd.DataFrame(data)

df_bike = simulate_bike_pricing_experiment()

# 区域级别分析
area_stats = df_bike.groupby('area_id').agg({
    'area_treatment': 'first',
    'weekly_rides': 'mean'
}).reset_index()

observed_ate = area_stats[area_stats['area_treatment'] == 1]['weekly_rides'].mean() - \
               area_stats[area_stats['area_treatment'] == 0]['weekly_rides'].mean()

print("📊 共享单车动态定价实验：")
print(f"\n观测结果：")
print(f"   - 实验组平均骑行: {area_stats[area_stats['area_treatment'] == 1]['weekly_rides'].mean():.1f} 次/周")
print(f"   - 对照组平均骑行: {area_stats[area_stats['area_treatment'] == 0]['weekly_rides'].mean():.1f} 次/周")
print(f"   - 观测 ATE: {observed_ate:.1f} 次/周")
print(f"\n⚠️  陷阱分析：")
print(f"   - 真实直接效应: -2.0 次/周（涨价的抑制作用）")
print(f"   - 负向溢出: +0.5 次/周（车更好找的正面作用）")
print(f"   - 观测效应 ≈ 直接效应 + 溢出效应 = -1.5 次/周")
print(f"\n💼 业务决策：")
print(f"   如果全量推广动态定价，所有区域都涨价")
print(f"   那么不存在"车更好找"的溢出效应")
print(f"   真实效应应该是 -2.0 次/周，比实验观测的更负！")

# 可视化
fig = go.Figure()

effects = ['观测 ATE', '真实直接效应', '溢出效应', '全量推广效应']
values = [observed_ate, -2.0, 0.5, -2.0]
colors = ['#95A5A6', '#EB5757', '#27AE60', '#E74C3C']

fig.add_trace(go.Bar(
    x=effects,
    y=values,
    marker_color=colors,
    text=[f'{v:.1f}' for v in values],
    textposition='outside'
))

fig.update_layout(
    title="共享单车定价实验：负向溢出导致效应被低估",
    yaxis_title="骑行次数变化（次/周）",
    height=400
)

fig.show()

---

## 练习与思考题

### 🤔 思考题 2

**场景**：抖音测试一个新的推荐算法，随机选择 50% 用户使用新算法。

**问题**：
1. 这个实验可能存在什么类型的网络效应？
2. 你会建议用什么实验设计？
3. 如果必须用个体随机化，如何分析溢出效应？

<details>
<summary>点击查看答案</summary>

**答案**：

1. **可能的网络效应**：
   - 正向溢出：实验组用户看到更好的内容，创作更多，对照组用户也能看到（通过关注关系）
   - 负向溢出：新算法给实验组用户推荐更多热门内容，占用流量池，对照组创作者曝光下降

2. **实验设计建议**：
   - **时间切片**：不同时段用不同算法（但要考虑时间效应）
   - **地理聚类**：不同地区用不同算法
   - **内容池分离**：实验组和对照组看不同的内容池（技术上复杂）

3. **个体随机化的分析**：
   - 构建用户关系图（关注/粉丝）
   - 计算每个用户的实验组关注比例
   - 使用 Ego-cluster 方法估计直接效应和溢出效应
   - 特别关注创作者用户，他们的粉丝分布会影响溢出效应

</details>

---

### 🤔 思考题 3

**场景**：美团测试"会员专享配送费减免"，给随机 30% 用户发放会员权益。

**问题**：
1. 为什么不能用个体随机化？
2. 如果用商圈聚类随机化，每个商圈需要多少用户才够？
3. 估计值会高估还是低估真实效应？为什么？

<details>
<summary>点击查看答案</summary>

**答案**：

1. **不能用个体随机化的原因**：
   - 骑手是共享资源，在同一商圈内服务所有用户
   - 实验组用户减免配送费 → 下单更多 → 骑手更忙 → 对照组用户配送时间变长
   - 负向溢出会导致效应估计偏差

2. **样本量计算**：
   - 需要估计聚类内相关系数 (ICC)，通常骑手资源的 ICC 在 0.1-0.3
   - 假设 ICC = 0.2，每个商圈至少需要 100-200 用户
   - 设计效应 = 1 + (m-1) × ICC ≈ 1 + 150 × 0.2 = 31
   - 相当于需要 31 倍的样本量！

3. **估计偏差方向**：
   - **会高估**真实效应
   - 原因：
     - 实验组：配送费减免的直接效应 - 骑手繁忙的负面影响
     - 对照组：0 - 骑手繁忙的负面影响
     - 观测 ATE = 直接效应（对照组也受到负面影响，基线被降低了）
   - 全量推广时，所有用户都会让骑手更忙，负面影响更大，真实效应更小

</details>

---

## 总结与要点

### 核心概念回顾

| 概念 | 定义 | 关键公式 |
|-----|------|----------|
| **SUTVA** | 无干扰假设 + 一致性假设 | $Y_i(T_i, T_{-i}) = Y_i(T_i)$ |
| **聚类随机化** | 以群组为单位随机化 | Design Effect = $1 + (m-1) \times \text{ICC}$ |
| **Ego Network** | 以用户为中心的局部网络 | Ego + Alters + Ties |
| **直接效应** | 用户自己接受处理的影响 | $\beta_1$ in $Y = \beta_0 + \beta_1 T + \beta_2 \bar{T}_{-i}$ |
| **溢出效应** | 用户好友接受处理的影响 | $\beta_2$ in $Y = \beta_0 + \beta_1 T + \beta_2 \bar{T}_{-i}$ |
| **总效应** | 全量推广的效应 | $\beta_1 + \beta_2$ |

---

### 方法选择决策树

```
是否存在网络效应？
├─ 否 → 标准 A/B 测试
└─ 是
    ├─ 能否定义清晰的聚类？
    │   ├─ 是
    │   │   ├─ 聚类数量 ≥ 20？
    │   │   │   ├─ 是 → 聚类随机化 ✅
    │   │   │   └─ 否 → 时间切片 / 地理分区
    │   └─ 否 → Ego-cluster 方法
    └─ 网络效应方向？
        ├─ 正向 → 实验效应被低估
        └─ 负向 → 实验效应被高估
```

---

### 实践建议

1. **实验设计阶段**：
   - 画出网络结构图，识别可能的干扰路径
   - 选择合适的聚类单位（地理、时间、社区等）
   - 计算所需的聚类数量和样本量

2. **数据收集阶段**：
   - 记录用户的社交关系（好友、关注等）
   - 记录用户的聚类信息（城市、商圈等）
   - 收集足够的协变量以控制混淆

3. **分析阶段**：
   - 使用聚类稳健标准误
   - 分别估计直接效应和溢出效应
   - 进行敏感性分析（不同的聚类定义）

4. **决策阶段**：
   - 考虑全量推广后的总效应，而非实验 ATE
   - 评估网络效应对长期效果的影响
   - 监控推广后的实际效果，验证预测

---

### 延伸阅读

**经典论文**：
1. Aronow, P. M., & Samii, C. (2017). "Estimating average causal effects under general interference"
2. Ugander, J., et al. (2013). "Graph cluster randomization: Network exposure to multiple universes" (Facebook)
3. Blake, T., & Coey, D. (2014). "Why marketplace experimentation is harder than it seems" (eBay)

**工业界实践**：
- Facebook: "Detecting network effects in A/B tests"
- LinkedIn: "Interference in online advertising experiments"
- Airbnb: "Measuring cross-side network effects"

---

### 下一步

恭喜你完成了网络效应的学习！你现在已经掌握了处理最复杂的实验场景之一。

**接下来可以学习**：
- 双边市场实验设计（供需两侧的平衡）
- 长期效应估计（网络效应需要时间传播）
- 因果图方法（DAG 识别干扰路径）

---

**🎉 本 Notebook 完成！**